In [30]:
#imports to keep
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select #to click with
from selenium.webdriver.common.by import By #to find with
from selenium.webdriver.support.wait import WebDriverWait

import chromedriver_binary

In [207]:
#surely shouldn't be hardcoding
URL = "https://www.google.com/search?q=roland+garros"
cookie_button_class_list = ["QS5gu", "sy4vM"] #"."join for driver - not sure for soup... #split?
score_class_list = "tsp-fr tsp-fs tsp-cpd tsp-cbd".split() #["tsp-fr", "tsp-fs", "tsp-cpd", "tsp cbd"] 
player_name_class_list = "tsp-nd tsp-db tsp-el".split() #pasted

In [173]:
def init_driver():
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

driver = init_driver()

In [174]:
def get_through_cookie_popup(driver):
    
    driver.get(URL)
    elements = driver.find_elements(By.CLASS_NAME, ".".join(cookie_button_class_list) )
    #print(elements)

    elem_to_click = None
    for e in elements:
        #print(e.text.lower())

        if "reject all" in e.text.lower():#.startswith("reject all"):
            elem_to_click = e

            
    if elem_to_click is not None:
        #now click the reject button
        print(f"clicked the {elem_to_click.text} button")
        driver.execute_script("arguments[0].click();", elem_to_click)
    else:
        return None
    
    return driver

driver = init_driver()
driver = get_through_cookie_popup(driver)

clicked the Reject all button


In [191]:
soup = BeautifulSoup(driver.page_source, "html.parser")
temp = soup.find_all("div", class_=score_class_list)
# print(len(temp))
#print(temp);

In [229]:
def get_all_scorecards(html):
    #score_class_list = ["tsp-fr", "tsp-fs", "tsp-cpd", "tsp cbd"]
    soup = BeautifulSoup(html, "html.parser")
    match_divs = soup.find_all("div", class_ =score_class_list)
    
    #every other div corresponds to a game, also ignores last text about times in uk time
    return match_divs[::2]

scorecards = get_all_scorecards(driver.page_source)
print(f"found {len(scorecards)} scorecards")
print(scorecards[0].prettify())

found 26 scorecards
<div class="tsp-fr tsp-fs tsp-cpd tsp-cbd" jsname="HNKBQd">
 <div class="tsp-led" role="button" tabindex="0">
  <div>
   <div>
    <div class="tsp-w tsp-dib tsp-fs tsp-eds">
     <div class="tsp-di">
      <div class="tsp-di">
       <span class="tsp-fm">
        First round
       </span>
       <span class="tsp-fl tsp-sp" role="separator">
        ·
       </span>
       <span class="tsp-lo">
        Philippe-Chatrier
       </span>
      </div>
     </div>
     <div class="tsp-sts tsp-pr">
      <span class="tsp-sis tsp-frs tsp-dnil tsp-lv" style="height:16px;line-height:16px;width:16px;fill:currentColor">
       <svg style="display:block;height:100%;width:100%" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
        <path d="M10 6L8.59 7.41 13.17 12l-4.58 4.59L10 18l6-6z">
        </path>
        <path d="M0 0h24v24H0z" fill="none">
        </path>
       </svg>
      </span>
      <span class="tsp-lv tsp-fm">
       Live
      </span>
      <div class="

In [227]:
def get_player_names_from_scorecard(score_html):
    names = []
    #name_class_list = "tsp-nd tsp-db tsp-el".split()

    for div in score_html:
        spans = div.find_all("span", class_ = player_name_class_list)
        for s in spans:
            print(s.contents[0])
            names.append(s.contents[0])
    return names


get_player_names_from_scorecard(scorecards[-4])

J.C. Shang
J. Varillas


['J.C. Shang', 'J. Varillas']

In [221]:
scorecards

[<div class="tsp-fr tsp-fs tsp-cpd tsp-cbd" jsname="HNKBQd"><div class="tsp-led" role="button" tabindex="0"><div><div><div class="tsp-w tsp-dib tsp-fs tsp-eds"><div class="tsp-di"><div class="tsp-di"><span class="tsp-fm">First round</span><span class="tsp-fl tsp-sp" role="separator">·</span><span class="tsp-lo">Philippe-Chatrier</span></div></div><div class="tsp-sts tsp-pr"><span class="tsp-sis tsp-frs tsp-dnil tsp-lv" style="height:16px;line-height:16px;width:16px;fill:currentColor"><svg style="display:block;height:100%;width:100%" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M10 6L8.59 7.41 13.17 12l-4.58 4.59L10 18l6-6z"></path><path d="M0 0h24v24H0z" fill="none"></path></svg></span><span class="tsp-lv tsp-fm">Live</span><div class="imso_li__live-ind-mask"><div class="imso_li__live-ind-wrap"><div class="imso_li__live-ind-var"></div></div></div></div></div></div></div><div class=""><table class="tsp-flr tsp-tpt tsp-tbl"><colgroup><col class="tsp-flc"/><col class="t

In [194]:
def get_all_data_from_scorecard_html():
    table_list = "tsp-flr tsp-tpt tsp-tbl".split()
    match_table= soup.find_all("table", class_ = table_list)
    #FINISH ME